In [11]:
from flask import Flask, render_template, request
import numpy as np
import re
import base64
from PIL import Image
from imageio import imread, imwrite
from keras.models import load_model
import json

In [12]:
def normalize(nparr):
    return np.interp(nparr, [0, 255], [-1,1])

In [13]:
app = Flask(__name__)

# mlp = load_model("./models/mlp_94.h5")
object_file = open("./object.txt", "r")
objects = object_file.readlines()
object_file.close()
N_CLASSES = len(objects)
CLASSES = {}
for idx, obj in enumerate(objects):
    CLASSES[idx] = obj.replace('\n', '')
print(CLASSES)
conv = load_model("./model/doodle.h5")
SHAPES = CLASSES

def save_numpy_array(array, file_path):
    """
    Save a NumPy array to a file.

    Parameters:
    - array: NumPy array to be saved.
    - file_path: File path to save the array to.
    """
    np.save(file_path, array)
    print(f"NumPy array saved to {file_path}")

@app.route("/", methods=["GET", "POST"])
def ready():
    if request.method == "GET":
        classes =list(SHAPES.values())
        classesText = f"Draw an object belong to one of the following categories: {', '.join(classes)}. Then see the prediction"
        return render_template("index1.html", classesText=classesText)
    if request.method == "POST":
        data = request.form["payload"].split(",")[1]
        net = request.form["net"]

        img = base64.decodebytes(data.encode('utf-8'))
        with open('static/temp.png', 'wb') as output:
            output.write(img)
        with open('temp.png', 'wb') as output:
            output.write(img)
        x = imread('temp.png', pilmode='L')
        # resize input image to 28x28
        x = np.array(Image.fromarray(x).resize((28, 28)))

        # if net == "MLP":
        #     model = mlp
        #     # invert the colors
        #     x = np.invert(x)
        #     # flatten the matrix
        #     x = x.flatten()

        if net == "CNN":
            model = conv
            x = np.expand_dims(x, axis=0)
            x = np.reshape(x, (28, 28, 1))
            # invert the colors
            x = np.invert(x)
            # brighten the image by 60%
            for i in range(len(x)):
                for j in range(len(x)):
                    if x[i][j] > 50:
                        x[i][j] = min(255, x[i][j] + x[i][j] * 0.60)

        # normalize the values between -1 and 1
        x = normalize(x)
        val = model.predict(np.array([x]))
        print(val)
        pred = SHAPES[np.argmax(val)]
        classes =list(SHAPES.values())
        print(pred)
        print(classes)
        print(list(val[0]))
        classesText = f"Draw an object belong to one of the following categories: {', '.join(classes)}. Then see the prediction"
        return render_template("index1.html", preds=list(val[0]), classes=json.dumps(classes), chart=True, putback=request.form["payload"], net=net, tempImage='./temp.png', classesText=classesText)

{0: 'apple', 1: 'banana', 2: 'grapes', 3: 'strawberry'}


In [14]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jan/2024 14:18:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2024 14:18:24] "GET /static/index.js HTTP/1.1" 304 -


1/1 [==============================] - 0s 81ms/step


C:\Users\Admin\AppData\Local\Temp\ipykernel_17540\970322433.py:41: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  x = imread('temp.png', pilmode='L')
127.0.0.1 - - [03/Jan/2024 14:18:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2024 14:18:46] "GET /static/index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Jan/2024 14:18:46] "GET /static/temp.png HTTP/1.1" 200 -


[[1.17524054e-04 5.43695269e-03 1.37869979e-03 9.93066788e-01]]
strawberry
['apple', 'banana', 'grapes', 'strawberry']
[0.000117524054, 0.0054369527, 0.0013786998, 0.9930668]
1/1 [==============================] - 0s 24ms/step


127.0.0.1 - - [03/Jan/2024 14:18:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2024 14:18:59] "GET /static/index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Jan/2024 14:18:59] "GET /static/temp.png HTTP/1.1" 200 -


[[4.1912785e-06 9.8626190e-01 3.4901185e-04 1.3384818e-02]]
banana
['apple', 'banana', 'grapes', 'strawberry']
[4.1912785e-06, 0.9862619, 0.00034901185, 0.013384818]
1/1 [==============================] - 0s 22ms/step


127.0.0.1 - - [03/Jan/2024 14:19:13] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2024 14:19:14] "GET /static/index.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Jan/2024 14:19:14] "GET /static/temp.png HTTP/1.1" 200 -


[[5.9845700e-04 2.4006269e-03 8.6420393e-01 1.3279703e-01]]
grapes
['apple', 'banana', 'grapes', 'strawberry']
[0.000598457, 0.002400627, 0.86420393, 0.13279703]
